<a href="https://colab.research.google.com/github/nilkanth02/pytorch-learnings/blob/main/mnist.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from torchvision.utils import make_grid

import numpy as np
import pandas as pd
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
%matplotlib inline



In [3]:
# converting mnist img files into tensor of 4 dimensions (# of imgs, height , width, color channel  )
transform = transforms.ToTensor()

In [4]:
# training the data
train_data = datasets.MNIST(root = '/cnn_data', train = True, download = True, transform = transform)


Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 9912422/9912422 [00:00<00:00, 17522265.96it/s]


Extracting /cnn_data/MNIST/raw/train-images-idx3-ubyte.gz to /cnn_data/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 28881/28881 [00:00<00:00, 483336.37it/s]


Extracting /cnn_data/MNIST/raw/train-labels-idx1-ubyte.gz to /cnn_data/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 1648877/1648877 [00:00<00:00, 4384244.49it/s]


Extracting /cnn_data/MNIST/raw/t10k-images-idx3-ubyte.gz to /cnn_data/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 4542/4542 [00:00<00:00, 3309681.86it/s]

Extracting /cnn_data/MNIST/raw/t10k-labels-idx1-ubyte.gz to /cnn_data/MNIST/raw



In [5]:
# test data
test_data = datasets.MNIST(root = '/cnn_data', train = False, download = True, transform = transform)


In [6]:
train_data

Dataset MNIST
    Number of datapoints: 60000
    Root location: /cnn_data
    Split: Train
    StandardTransform
Transform: ToTensor()

In [7]:
test_data

Dataset MNIST
    Number of datapoints: 10000
    Root location: /cnn_data
    Split: Test
    StandardTransform
Transform: ToTensor()

In [11]:
# creating a small batch for images .....let's say 10
train_loader = DataLoader(train_data, batch_size=10, shuffle =True )
test_loader = DataLoader(test_data, batch_size=10, shuffle =False )

In [12]:
# define our CNN model
# describing convolutional layer and what it's doing (2 convo layers)
#
conv1 = nn.Conv2d(1, 6, 3, 1)
conv2 = nn.Conv2d(6, 16, 3, 1)

In [13]:
# grabing one mnist recird/img
for i, (X_train, y_train) in enumerate(train_data):
  break

In [16]:
X_train.shape


torch.Size([1, 28, 28])

In [17]:
x = X_train.view(1, 1, 28, 28)

In [18]:
# perfrom first convolution
x = F.relu(conv1(x)) #rectified linear unit for activation function


In [21]:
# 1 single image, 6 filters we asked for , 26x26
x.shape

torch.Size([1, 6, 26, 26])

In [22]:
# pass thru the pooling layer
x= F.max_pool2d(x, 2, 2)  # kernel of 2 and stride of 2

In [25]:
x.shape #26/2 =13

torch.Size([1, 6, 13, 13])

In [26]:
# second convo layer
x = F.relu(conv2(x))

In [27]:
x.shape # Again, we didn't set the padding around the outside of the image


torch.Size([1, 16, 11, 11])

In [28]:
# Pooling layer
x = F.max_pool2d(x, 2, 2)

In [29]:
x.shape # 11/2 = 5.5

torch.Size([1, 16, 5, 5])

In [30]:
# Model class
class ConvolutionalNetwork(nn.Module):
  def __init__(self):
    super().__init__()
    self.conv1 = nn.Conv2d(1, 6, 3, 1)
    self.conv2 = nn.Conv2d(6, 16, 3, 1)
    # fully connected layer
    self.fc1 = nn.Linear(5*5*16, 120)
    self.fc2 = nn.Linear(120, 84)
    self.fc3 = nn.Linear(84, 10) # 10 classes

  def forward(self, X):
    X = F.relu(self.conv1(X))
    X = F.max_pool2d(X, 2, 2)
    # second pass
    X = F.relu(self.conv2(X))
    X = F.max_pool2d(X, 2, 2)

    # re-view to flatten it out
    X = X.view(-1, 16*5*5) #negative one so that we can vary the batch size

    # Fully connected layers
    X = F.relu(self.fc1(X))
    X = F.relu(self.fc2(X))
    X = self.fc3(X)
    return F.log_softmax(X, dim=1)


In [31]:
# Create an instance fro our model
torch.manual_seed(41)
model = ConvolutionalNetwork()
model

ConvolutionalNetwork(
  (conv1): Conv2d(1, 6, kernel_size=(3, 3), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(3, 3), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)

In [33]:
# Loss function optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
